#### Langchain, openai key and HUggingFace API key

In [19]:
from langchain_openai import ChatOpenAI
from constant import openai_key
from huggingface_Api import HUGGINGFACEHUB_API_TOKEN

#### Call the openai env key globally

In [7]:
import os
os.environ["OPENAI_API_KEY"] = openai_key

#### Initialize the OpenAI LLM via Langchain

In [8]:
llm = ChatOpenAI(temperature=0.8) # More the value towards 1 more random the LLM would be

##### Testing

In [10]:
text = " what is the capital of india"
print(llm.predict(text)) # also we can use (llm.invoke)

The capital of India is New Delhi.


#### From Huggingface 
###### We can use opensource model from Huggingface

In [15]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN